In [1]:
import os
import re
import json
import requests
from datetime import datetime
from typing import Any
from collections import defaultdict

from bs4 import BeautifulSoup, Tag
from urllib.parse import urljoin, unquote

BASE_URL = 'https://vi.wikipedia.org'
MAIN_URL = BASE_URL + '/wiki/Wikipedia:B%E1%BA%A1n_c%C3%B3_bi%E1%BA%BFt'

OUTPUT_DIR = 'data/vi'

NUMBER_TO_MONTH = {
    1: 'Tháng một',
    2: 'Tháng hai',
    3: 'Tháng ba',
    4: 'Tháng tư',
    5: 'Tháng năm',
    6: 'Tháng sáu',
    7: 'Tháng bảy',
    8: 'Tháng tám',
    9: 'Tháng chín',
    10: 'Tháng mười',
    11: 'Tháng mười một',
    12: 'Tháng mười hai',
}


def calculate_month_number_from_week_number(week_number: int) -> int:
    """
    Calculate the approximate month number from the week number.
    """
    return (week_number - 1) * 7 // 30 + 1


def get_year_links_from_archive(main_page_url: str) -> dict[str, dict[str, Any]]:
    """
    Get the year links from the main archive page.
    Current year link is the main page.
    """
    resp = requests.get(main_page_url)
    resp.raise_for_status()
    soup = BeautifulSoup(resp.text, 'html.parser')

    div = soup.find('div', class_='hlist')
    if not div:
        raise ValueError("Could not find the div.")

    archive_links = {}
    for a_tag in div.find_all('a', href=True):        
        year = a_tag.text.strip()
        if re.match(r'^\d{4}$', year):
            href = a_tag['href']
            full_url = urljoin(BASE_URL, href)
            exists = 'new' not in a_tag.get('class', [])
            archive_links[year] = {
                'url': full_url,
                'exists': exists
            }

    return archive_links

In [2]:
archive = get_year_links_from_archive(MAIN_URL)

archive

{'2006': {'url': 'https://vi.wikipedia.org/wiki/Wikipedia:B%E1%BA%A1n_c%C3%B3_bi%E1%BA%BFt/2006',
  'exists': True},
 '2007': {'url': 'https://vi.wikipedia.org/wiki/Wikipedia:B%E1%BA%A1n_c%C3%B3_bi%E1%BA%BFt/2007',
  'exists': True},
 '2008': {'url': 'https://vi.wikipedia.org/wiki/Wikipedia:B%E1%BA%A1n_c%C3%B3_bi%E1%BA%BFt/2008',
  'exists': True},
 '2009': {'url': 'https://vi.wikipedia.org/wiki/Wikipedia:B%E1%BA%A1n_c%C3%B3_bi%E1%BA%BFt/2009',
  'exists': True},
 '2010': {'url': 'https://vi.wikipedia.org/wiki/Wikipedia:B%E1%BA%A1n_c%C3%B3_bi%E1%BA%BFt/2010',
  'exists': True},
 '2011': {'url': 'https://vi.wikipedia.org/wiki/Wikipedia:B%E1%BA%A1n_c%C3%B3_bi%E1%BA%BFt/2011',
  'exists': True},
 '2012': {'url': 'https://vi.wikipedia.org/wiki/Wikipedia:B%E1%BA%A1n_c%C3%B3_bi%E1%BA%BFt/2012',
  'exists': True},
 '2013': {'url': 'https://vi.wikipedia.org/wiki/Wikipedia:B%E1%BA%A1n_c%C3%B3_bi%E1%BA%BFt/2013',
  'exists': True},
 '2014': {'url': 'https://vi.wikipedia.org/wiki/Wikipedia:B%E1%B

In [17]:
def _extract_fact_data(element: Tag, base_url: str) -> dict[str, Any]:
    """Extracts text, links, and relevant links from a BeautifulSoup Tag."""
    fact_text = "Bạn có biết " + element.get_text(" ", strip=True).lstrip().lstrip('.').lstrip('…').lstrip()
    fact_text = fact_text.replace('\xa0', ' ').replace('  ', ' ')

    links = []
    relevant_links = []
    for a in element.find_all("a", href=True):
        href = a["href"]
        if not href.startswith("/wiki/"):
            continue

        full_url = unquote(urljoin(base_url, href))

        if a.find_parent('b'):
            relevant_links.append(full_url)

        links.append(full_url)

    return {
        "text": fact_text,
        "links": links,
        "relevant_links": relevant_links,
    }


def parse_year_facts(year_url: str) -> list[dict]:
    """Parse the facts from the year page."""
    resp = requests.get(year_url)
    resp.raise_for_status()
    soup = BeautifulSoup(resp.text, "html.parser")

    results: list[dict] = []

    # Parse facts from all uls in section div
    div = soup.find("div", class_="mw-content-ltr mw-parser-output")
    if not div:
        raise ValueError("Could not find the div.")

    table = div.find('table')
    if not table:
        raise ValueError("Could not find the table.")

    for tr in table.find_all('tr'):
        for td in tr.find_all('td'):
            try:
                section = td.find('h3').text.strip()
            except Exception:
                continue

            ul = td.find('ul')
            for li in ul.find_all('li'):
                fact_data = _extract_fact_data(li, year_url)

                if fact_data['text']:
                    results.append({
                        "section": section,
                        **fact_data
                    })
    
    return results

In [19]:
year = '2025'

facts = parse_year_facts(archive[year]['url'])
facts

[{'section': 'Tuần 1 lần 1',
  'text': 'Bạn có biết nữ diễn viên Trung Quốc Vương Đan Phượng từng được mời tham dự lễ nhậm chức của tổng thống Mỹ Ronald Reagan ?',
  'links': ['https://vi.wikipedia.org/wiki/Trung_Quốc',
   'https://vi.wikipedia.org/wiki/Vương_Đan_Phượng',
   'https://vi.wikipedia.org/wiki/Tổng_thống_Hoa_Kỳ',
   'https://vi.wikipedia.org/wiki/Ronald_Reagan'],
  'relevant_links': ['https://vi.wikipedia.org/wiki/Vương_Đan_Phượng']},
 {'section': 'Tuần 1 lần 1',
  'text': 'Bạn có biết British Steel của ban nhạc Judas Priest từng được xem là album định nghĩa thể loại heavy metal ?',
  'links': ['https://vi.wikipedia.org/wiki/British_Steel_(album)',
   'https://vi.wikipedia.org/wiki/Judas_Priest',
   'https://vi.wikipedia.org/wiki/Heavy_metal'],
  'relevant_links': ['https://vi.wikipedia.org/wiki/British_Steel_(album)',
   'https://vi.wikipedia.org/wiki/Judas_Priest']},
 {'section': 'Tuần 1 lần 1',
  'text': 'Bạn có biết chỉ với kinh phí từ 150 – 200 triệu đồng , bộ phim Ngư

In [53]:
MONTHS = {
    'січень': 'січень', 'січня': 'січень',
    'лютий': 'лютий', 'лютого': 'лютий',
    'березень': 'березень', 'березня': 'березень',
    'квітень': 'квітень', 'квітня': 'квітень',
    'травень': 'травень', 'травня': 'травень',
    'червень': 'червень', 'червня': 'червень',
    'липень': 'липень', 'липня': 'липень',
    'серпень': 'серпень', 'серпня': 'серпень',
    'вересень': 'вересень', 'вересня': 'вересень',
    'жовтень': 'жовтень', 'жовтня': 'жовтень',
    'листопад': 'листопад', 'листопада': 'листопад',
    'грудень': 'грудень', 'грудня': 'грудень',
}

def extract_month_from_title(title: str) -> str | None:
    """Extracts the first mentioned month from a section title."""
    words = re.split(r'[\s—-]+', title)

    for word in words:
        cleaned_word = word.lower().strip()
        
        if cleaned_word in MONTHS:
            return MONTHS[cleaned_word]

    return None

In [60]:
extract_month_from_title('3 22 грудня 2022 — 23 січня 2023')

'грудень'

In [52]:
import dateparser

year = '2025'
month = '7'
section = '11 — 25 лютого 2025'

dt_month = dateparser.parse(section)
fact_date = dt_month.strftime("%Y-%m") if dt_month else None

fact_date

'2025-02'

In [1]:
a = '2015-07-27'
a.split('-')

['2015', '07', '27']

In [ ]:
''